# Сравнение предсказаний нескольких SR-моделей

Этот ноутбук строит **один общий montage** по одинаковому набору изображений, но с колонками SR (и SR-кропов) для **нескольких моделей**.

Поддерживаются два типовых лэйаута папок внутри `model_root`:

**Вариант 1 (подпапки):**
```
model_root/
  lr/  *.png
  hr/  *.png
  sr/  *.png
```

**Вариант 2 (плоско, суффиксы):**
```
model_root/
  ..._lr.png / ..._LR.png
  ..._hr.png / ..._HR.png
  ..._sr.png / ..._SR.png
```

> LR и HR берутся из **первой** модели (считаем, что они одинаковы), а SR — из каждой модели.


In [4]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageChops, ImageOps
import re
from typing import Dict, List, Tuple, Optional

# =======================
# ПАРАМЕТРЫ
# =======================
MODELS = [
    {"name": "carbonates_only_model", "root": Path("../../preds/preds_test1_carbonates_x4_msresunet")},
    {"name": "all_rounded_model", "root": Path("../../preds/preds_test3_x4_msresunet")},
]
OUT_PATH = Path("../../preds_compare_montage_x4.png")

# 1) ВАРИАНТ A: box относительно центра (ширина/высота в пикселях)
BOX_SIZE   = (100, 100)   # (w, h)
BOX_OFFSET = (0, 0)       # (dx, dy) от центра

# 2) ВАРИАНТ B: точные координаты (x0, y0, x1, y1)
USE_ABSOLUTE_BOX = False
ABS_BOX = (50, 50, 150, 150)

# Размер отображения "full" (если None — не ресайзить)
TARGET_FULL_SIZE = (500, 500)

# Размер отображения crop-тайлов (если None — не ресайзить кропы)
CROP_DISPLAY_SIZE = (250, 250)   # сделай (300, 300) / (400, 400) если надо крупнее
# Альтернатива: множитель (если хочешь апскейл в N раз без ручного размера)
CROP_UPSCALE_FACTOR = None       # например 3 или 4

# Включить difference tiles (|SR - HR|) для каждой модели
SHOW_DIFF = True

# -----------------------
# Визуальные параметры
# -----------------------
PADDING = 8
LABEL_HEIGHT = 24
BG = (20, 20, 20)
FG = (235, 235, 235)
RED = (220, 40, 40)
RESAMPLE = Image.BICUBIC

# =======================
# ВСПОМОГАТЕЛЬНОЕ
# =======================
def _try_load_font(size=16):
    # максимально переносимый вариант: PIL поищет системный шрифт
    for name in ["DejaVuSans.ttf", "arial.ttf"]:
        try:
            return ImageFont.truetype(name, size=size)
        except Exception:
            pass
    return ImageFont.load_default()

FONT = _try_load_font(16)

def _to_rgb(img: Image.Image) -> Image.Image:
    if img.mode == "RGB":
        return img
    if img.mode in ("I;16", "I"):
        # 16-bit / int -> нормируем в 8-bit для визуализации
        arr = img
        # PIL сам не умеет "умно" нормировать I;16 -> L без numpy, поэтому:
        # используем простую линейную конвертацию через point с клипом.
        # (обычно уже до 8bit у тебя предсказания; этот блок на всякий случай)
        img = img.convert("I")
        img8 = img.point(lambda v: max(0, min(255, int(v / 256))))
        return img8.convert("RGB")
    return img.convert("RGB")

def load_image(path: Path) -> Image.Image:
    img = Image.open(path)
    return _to_rgb(img)

def resize_if_needed(img: Image.Image, target_size: Optional[Tuple[int,int]]) -> Image.Image:
    if target_size is None:
        return img
    if img.size == target_size:
        return img
    return img.resize(target_size, RESAMPLE)

def resize_crop_for_display(img: Image.Image) -> Image.Image:
    if CROP_UPSCALE_FACTOR is not None:
        w, h = img.size
        return img.resize((w * CROP_UPSCALE_FACTOR, h * CROP_UPSCALE_FACTOR), RESAMPLE)
    return resize_if_needed(img, CROP_DISPLAY_SIZE)

def compute_center_box(w: int, h: int, box_size: Tuple[int,int], offset: Tuple[int,int]) -> Tuple[int,int,int,int]:
    bw, bh = box_size
    cx, cy = w // 2 + offset[0], h // 2 + offset[1]
    x0 = max(0, cx - bw // 2)
    y0 = max(0, cy - bh // 2)
    x1 = min(w, x0 + bw)
    y1 = min(h, y0 + bh)
    # если упёрлись в край, сдвинем назад чтобы размер сохранился
    x0 = max(0, x1 - bw)
    y0 = max(0, y1 - bh)
    return (x0, y0, x1, y1)

def make_tile(img: Image.Image, label: str) -> Image.Image:
    img = _to_rgb(img)
    w, h = img.size
    tile = Image.new("RGB", (w + PADDING*2, h + PADDING*2 + LABEL_HEIGHT), BG)
    tile.paste(img, (PADDING, PADDING))
    d = ImageDraw.Draw(tile)
    d.text((PADDING, h + PADDING + 3), label, font=FONT, fill=FG)
    return tile

def make_orig_tile_with_box(hr: Image.Image, box: Tuple[int,int,int,int], label: str, target_h: int) -> Image.Image:
    hr = _to_rgb(hr)
    prev = hr.copy()
    d = ImageDraw.Draw(prev)
    d.rectangle(box, outline=RED, width=3)
    prev = resize_if_needed(prev, TARGET_FULL_SIZE)
    tile = make_tile(prev, label)
    # согласуем высоту (если нужно)
    if tile.size[1] != target_h:
        tile = tile.resize((tile.size[0], target_h), RESAMPLE)
    return tile

def hstack(images: List[Image.Image]) -> Image.Image:
    widths = [im.size[0] for im in images]
    heights = [im.size[1] for im in images]
    out = Image.new("RGB", (sum(widths), max(heights)), BG)
    x = 0
    for im in images:
        out.paste(im, (x, 0))
        x += im.size[0]
    return out

def vstack(images: List[Image.Image]) -> Image.Image:
    widths = [im.size[0] for im in images]
    heights = [im.size[1] for im in images]
    out = Image.new("RGB", (max(widths), sum(heights)), BG)
    y = 0
    for im in images:
        out.paste(im, (0, y))
        y += im.size[1]
    return out

def diff_tile(sr: Image.Image, hr: Image.Image, label: str) -> Image.Image:
    # |SR - HR|, усиление контраста для наглядности
    sr = _to_rgb(sr)
    hr = _to_rgb(hr)
    if sr.size != hr.size:
        hr = hr.resize(sr.size, RESAMPLE)
    # abs diff по каналам
    diff = ImageChops.difference(sr, hr)
    # усиление: растянем гистограмму
    diff = ImageOps.autocontrast(diff)
    return make_tile(diff, label)

# PIL modules для diff
from PIL import ImageChops, ImageOps

# =======================
# ПОИСК ФАЙЛОВ
# =======================
def _index_by_stem(folder: Path) -> Dict[str, Path]:
    out = {}
    for p in folder.rglob("*"):
        if p.is_file() and p.suffix.lower() in [".png", ".jpg", ".jpeg", ".tif", ".tiff"]:
            out[p.stem] = p
    return out

def _detect_layout(root: Path) -> str:
    # "subfolders" если есть lr/hr/sr директории
    lr = (root / "lr")
    hr = (root / "hr")
    sr = (root / "sr")
    if lr.exists() and hr.exists() and sr.exists():
        return "subfolders"
    return "flat"

def discover_samples(model_root: Path) -> Dict[str, Dict[str, Path]]:
    """
    Возвращает mapping:
      sample_id -> {"lr": path, "hr": path, "sr": path}
    """
    layout = _detect_layout(model_root)
    if layout == "subfolders":
        lr_idx = _index_by_stem(model_root / "lr")
        hr_idx = _index_by_stem(model_root / "hr")
        sr_idx = _index_by_stem(model_root / "sr")
        keys = sorted(set(lr_idx) & set(hr_idx) & set(sr_idx))
        return {k: {"lr": lr_idx[k], "hr": hr_idx[k], "sr": sr_idx[k]} for k in keys}

    # flat layout: ищем по суффиксам в имени
    files = [p for p in model_root.rglob("*") if p.is_file() and p.suffix.lower() in [".png", ".jpg", ".jpeg", ".tif", ".tiff"]]
    # нормализуем стем: удаляем _lr/_hr/_sr (в разных регистрах)
    def norm(stem: str) -> Tuple[Optional[str], Optional[str]]:
        m = re.match(r"^(.*?)(?:[_\-](lr|hr|sr))$", stem, flags=re.IGNORECASE)
        if m:
            return m.group(1), m.group(2).lower()
        # иногда бывают ..._pred вместо sr
        m = re.match(r"^(.*?)(?:[_\-](pred|srpred))$", stem, flags=re.IGNORECASE)
        if m:
            return m.group(1), "sr"
        return None, None

    tmp: Dict[str, Dict[str, Path]] = {}
    for p in files:
        base, kind = norm(p.stem)
        if base is None:
            continue
        tmp.setdefault(base, {})[kind] = p
    out = {k:v for k,v in tmp.items() if all(x in v for x in ["lr","hr","sr"])}
    return dict(sorted(out.items(), key=lambda kv: kv[0]))

def build_global_index(models: List[Dict]) -> Tuple[List[str], List[Dict[str, Dict[str, Path]]]]:
    per_model = []
    keys = None
    for m in models:
        idx = discover_samples(m["root"])
        per_model.append(idx)
        kset = set(idx.keys())
        keys = kset if keys is None else (keys & kset)  # пересечение, чтобы не падать
    common = sorted(keys) if keys is not None else []
    return common, per_model

# =======================
# СБОРКА MONTAGE
# =======================
def build_montage():
    common_keys, per_model = build_global_index(MODELS)
    if not common_keys:
        raise RuntimeError("Не нашёл ни одного общего sample_id между моделями. Проверь структуру папок/имена файлов.")

    rows = []

    for sample_id in common_keys:
        # берем LR/HR из первой модели (как "референс")
        ref = per_model[0][sample_id]
        lr = resize_if_needed(load_image(ref["lr"]), TARGET_FULL_SIZE)
        hr = resize_if_needed(load_image(ref["hr"]), TARGET_FULL_SIZE)

        # box на HR
        if USE_ABSOLUTE_BOX:
            box = ABS_BOX
        else:
            box = compute_center_box(hr.size[0], hr.size[1], BOX_SIZE, BOX_OFFSET)

        # full tiles: LR, HR, SR_i...
        tiles_full = [
            make_tile(lr, f"{sample_id} | LR"),
            make_tile(hr, f"{sample_id} | HR"),
        ]

        sr_full_imgs = []
        for mi, m in enumerate(MODELS):
            sr_path = per_model[mi][sample_id]["sr"]
            sr = resize_if_needed(load_image(sr_path), TARGET_FULL_SIZE)
            sr_full_imgs.append(sr)
            tiles_full.append(make_tile(sr, f"{m['name']} | SR"))

        # HR preview with box
        tiles_full.append(make_orig_tile_with_box(hr, box, "HR | crop box", target_h=tiles_full[0].size[1]))

        row_full = hstack(tiles_full)
        rows.append(row_full)

        lr_crop_raw = lr.crop(box)
        hr_crop_raw = hr.crop(box)
        
        lr_c = resize_crop_for_display(lr_crop_raw)
        hr_c = resize_crop_for_display(hr_crop_raw)
        
        tiles_crop = [
            make_tile(lr_c, "LR | crop"),
            make_tile(hr_c, "HR | crop"),
        ]
        
        for m, sr in zip(MODELS, sr_full_imgs):
            sr_crop_raw = sr.crop(box)
            sr_c = resize_crop_for_display(sr_crop_raw)
            tiles_crop.append(make_tile(sr_c, f"{m['name']} | SR | crop"))
        
            if SHOW_DIFF:
                d = ImageChops.difference(_to_rgb(sr_crop_raw), _to_rgb(hr_crop_raw))
                d = ImageOps.autocontrast(d)
                d = resize_crop_for_display(d)
                tiles_crop.append(make_tile(d, f"{m['name']} | |SR-HR|"))

        row_crop = hstack(tiles_crop)
        rows.append(row_crop)

    montage = vstack(rows)
    OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    montage.save(OUT_PATH)
    print("Готово:", OUT_PATH.resolve())
    print("Samples:", len(common_keys))
    return OUT_PATH

build_montage()

Готово: C:\Users\Вячеслав\Documents\superresolution\preds_compare_montage_x4.png
Samples: 14


WindowsPath('../../preds_compare_montage_x4.png')